# Deep Q-Learning para _lunar lander_

[**Juan Gómez Romero**](https://decsai.ugr.es/~jgomez)  
Departamento de Ciencias de la Computación e Inteligencia Artificial  
Universidad de Granada  
This work is licensed under the [GNU General Public License v3.0](https://choosealicense.com/licenses/gpl-3.0/).

---
Ejemplo basado en:
> Udacity (2019) Deep Reinforcement Learning Course. Disponible en [GitHub](https://github.com/udacity/deep-reinforcement-learning/tree/master/dqn).

Comprobar si está ejecutando en Google Colaboratory.

## Explorar entorno virtual

En este ejercicio utilizamos el entorno virtual [LunarLander-v2](https://gym.openai.com/envs/LunarLander-v2/) de [OpenAI](https://openai.com). 

![](https://github.com/jgromero/eci2019-DRL/blob/master/Tema%204%20-%20Aprendizaje%20Profundo%20por%20Refuerzo/code/dqn/lunarlander.gif?raw=true)

En primer lugar, vamos a explorar cómo funciona este entorno.

In [1]:
import gfootball.env as football_env
from dqn_agent_simultaneous_update import Agent
from gfootball.env import football_action_set

env = football_env.create_environment(
    env_name='academy_run_to_score_with_keeper', 
    stacked=False,                           # solo estado, no pixeles 
    representation='simple115',              # solo estado, no pixeles 
    rewards='scoring,checkpoints',           # recompensas intermedias, no solo al marcar Currently supported rewards are 'scoring' and 'checkpoints'.
    render=True)                            # mostrar graficamente

agent = Agent(state_size=115, action_size=21, seed=0)

/home/luciano/anaconda3/envs/football/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/luciano/anaconda3/envs/football/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/luciano/anaconda3/envs/football/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/luciano/anaconda3/envs/football

## Algoritmo
A continuación se proporciona una implementación genérica del algoritmo Deep Q-Learning (DQN) y su aplicación a [LunarLander-v2](https://gym.openai.com/envs/LunarLander-v2/).

Se considera que el entorno [LunarLander-v2](https://gym.openai.com/envs/LunarLander-v2/) está resuelto cuando se obtienen más de $200$ puntos de media durante 100 episodios consecutivos.

In [2]:
import logging, os
#logging.disable(logging.WARNING)

In [3]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

### Visualizar agente entrenado

Podemos visualizar el comportamiento del agente entrenado cargando los pesos del fichero donde se han almacenado. (Solo en entorno local, no en Google Collaboratory.)

In [4]:
# cargar pesos del fichero `checkpoint.pth`
agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))

for i in range(10):
    state = env.reset()
    for j in range(1000):
        action = agent.act(state)
        state, reward, done, _ = env.step(action)
        if done:
            break 
            
env.close()

I0819 19:01:10.146096 140450344777536 observation_processor.py:350] Dump "lost_score": count limit reached / disabled
I0819 19:01:10.148620 140450344777536 observation_processor.py:350] Dump "episode_done": count limit reached / disabled
I0819 19:01:10.149393 140450344777536 football_env_wrapper.py:84] Episode reward: -1.00 score: [0, 1], steps: 45, FPS: 3.7, gameFPS: 37.3
I0819 19:01:12.982569 140450344777536 observation_processor.py:350] Dump "lost_score": count limit reached / disabled
I0819 19:01:12.983698 140450344777536 observation_processor.py:350] Dump "episode_done": count limit reached / disabled
I0819 19:01:12.984191 140450344777536 football_env_wrapper.py:84] Episode reward: -1.00 score: [0, 1], steps: 44, FPS: 15.5, gameFPS: 29.8
I0819 19:01:16.085605 140450344777536 observation_processor.py:350] Dump "lost_score": count limit reached / disabled
I0819 19:01:16.086936 140450344777536 observation_processor.py:350] Dump "episode_done": count limit reached / disabled
I0819 19: